In [1]:
import scipy.io as sio
import tensorflow as tf
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    ConvLSTM2D,
    GlobalMaxPooling2D, 
    )

In [2]:
mat = sio.loadmat('../../data/processed/sevenThirty/s01.mat')
mi = mat['mi']
real = mat['real']
class_mi = mat['class_mi']
class_real = mat['class_real']

In [3]:
mi_tensor = np.empty([mi.shape[2], 64, 1024])
real_tensor = np.empty([real.shape[2], 64, 1024])

In [8]:
mi_intermediate = np.empty([mi.shape[2], 16, 64, 64])


In [5]:
mi_intermediate.shape

(200, 4, 64, 256)

In [4]:
for i in range(0, mi.shape[2]):
    mi_tensor[i, :, :] = mi[:, :, i]

mi_intermediate = np.empty([mi.shape[2], 4, 64, 256])

for i in range(1, (mi.shape[2] - 1)):
    
    startIDX = (8 * i) - 1
    startIDX = startIDX - 4
    endIDX = startIDX + 16
    
mi_tensor = tf.reshape(mi_tensor, shape = (mi.shape[2], 64, 1024, 1))

In [5]:
for i in range(0, real.shape[2]):
    real_tensor[i, :, :] = real[:, :, i]
    
real_tensor = tf.reshape(real_tensor, shape = (real.shape[2], 64, 1024, 1))

In [6]:
class_mi = tf.reshape(class_mi, mi.shape[2])
class_real = tf.reshape(class_real, real.shape[2])

In [13]:
opt = tf.keras.optimizers.RMSprop(
    lr=0.001)

loss = tf.keras.losses.BinaryCrossentropy()

In [12]:
model = Sequential()
model.add(ConvLSTM2D(
    filters=4,
    kernel_size=(4, 4),
    padding='same',
    data_format='channels_last',
    return_sequences=False,
#     input_shape=(64, 1024, 1)
    ))
model.add(GlobalMaxPooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=['accuracy'])

In [16]:
model.fit(
    x=mi_tensor,
    y=class_mi,
    batch_size=1,
    validation_data=(real_tensor, class_real),
    epochs=10,
    verbose=2)

Epoch 1/10


ValueError: in user code:

    /home/swagmaster/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/swagmaster/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/swagmaster/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/swagmaster/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/swagmaster/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /hom

In [9]:
model_cnn_basic = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(128, (4,4), activation='relu', input_shape=(64, 1024, 1)),
  tf.keras.layers.MaxPooling2D(3, 3),
  tf.keras.layers.Conv2D(128, (4,4), activation='relu'),
  tf.keras.layers.MaxPooling2D(3,3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [10]:
model_cnn_basic.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [11]:
tf.reshape(new_mat, shape = (200, 64, 1024, 1))

<tf.Tensor: shape=(200, 64, 1024, 1), dtype=float64, numpy=
array([[[[-9.09148312e+00],
         [-9.18334103e+00],
         [-9.13973331e+00],
         ...,
         [-8.90516663e+00],
         [-8.84207630e+00],
         [-8.71076107e+00]],

        [[-1.06371918e+02],
         [-1.07051544e+02],
         [-1.06715981e+02],
         ...,
         [-1.05157043e+02],
         [-1.04579742e+02],
         [-1.03482391e+02]],

        [[-5.67636108e+02],
         [-5.70092834e+02],
         [-5.68882690e+02],
         ...,
         [-5.63805115e+02],
         [-5.61303833e+02],
         [-5.56863708e+02]],

        ...,

        [[ 4.56447852e+04],
         [ 8.72393359e+04],
         [ 4.82432930e+04],
         ...,
         [ 1.31842148e+04],
         [ 6.85449688e+04],
         [-1.93912305e+04]],

        [[ 3.85506484e+04],
         [ 8.26777812e+04],
         [ 4.24533789e+04],
         ...,
         [ 4.46134180e+03],
         [ 6.29898047e+04],
         [-2.59011992e+04]],

      

In [12]:
np.array(mat['class_mi'])[0] - 1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=uint8)

In [13]:
x = tf.reshape(new_mat, shape = (200, 64, 1024, 1))
y = tf.reshape(mat['class_mi'] , 200)

In [14]:
y

<tf.Tensor: shape=(200,), dtype=uint8, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=uint8)>

In [15]:
model_cnn_basic.fit(x, y,verbose = 1)

7/7 [==============================] - 18s 2s/step - loss: 237893.3278


In [16]:
tf.reshape(mat['class_mi'], 200)

<tf.Tensor: shape=(200,), dtype=uint8, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=uint8)>

In [17]:
mat['class_mi'].transpose().shape

(200, 1)